# Carbon Intensity Data Generator

In [7]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import sys

In [8]:
headers = {
  'Accept': 'application/json'
}

In [9]:
start_date = datetime(2022,1,1)
end_date = datetime(2023,1,1)


day_count = 0

df = pd.DataFrame()
df["group"] = []
df["ref value"] = []
df["day"] = []
df["hour"] = []
df["carbon_intensity"] = []

while start_date < end_date:
    day_count += 1
    hour_count  = 0 
    for hour in range(1,25):
        
        #request
        date = str(start_date.strftime('%Y-%m-%d'))
        period = str(hour*2)
        request = "https://api.carbonintensity.org.uk/intensity/date/"+date+"/"+period
        #print(request)
        response_carbon_intensity = requests.get(request, params={}, headers = headers) 
        #print(response_carbon_intensity.status_code)
        
        #Error Check
        if response_carbon_intensity.status_code != 200:
            print("Error: "+ str(response_carbon_intensity.json()))
            print("Date = "+ date)
            print("period = "+ period)
            hour_carbon_intenisty = np.nan
        else:    
            #print(response_carbon_intensity.json()["data"][0]["intensity"]["actual"])
            try:
                hour_carbon_intenisty = response_carbon_intensity.json()["data"][0]["intensity"]["actual"]
            except IndexError:
                print("Error: "+ str(response_carbon_intensity.json()))
                print("Date = "+ date)
                print("period = "+ period)
                hour_carbon_intenisty = np.nan

        #Append response
        datetime_value = "d_"+str(day_count-1).zfill(3)+"_h_"+str(hour-1).zfill(2)
        row = {"group":datetime_value,"ref value":hour_carbon_intenisty,"day": day_count-1,"hour": hour-1,"carbon_intensity": hour_carbon_intenisty}
        df = df.append(row,ignore_index=True)

    start_date += timedelta(days=1)
    
df = df.interpolate()

#print("\n Number of hours = " + str(hour_count))

Error: {'data': []}
Date = 2022-03-27
period = 48


In [10]:
#Check interpolation has gone right
differnce = datetime(2022,1,1) - datetime(2022,3,27)
print(differnce)

sizes = df.groupby(["day"]).size()
print(sizes[sizes!=24])

df[df["day"] ==85]

-85 days, 0:00:00
Series([], dtype: int64)


,group,ref value,day,hour,carbon_intensity
2040,d_085_h_00,211,85.0,0.0,211
2041,d_085_h_01,216,85.0,1.0,216
2042,d_085_h_02,214,85.0,2.0,214
2043,d_085_h_03,213,85.0,3.0,213
2044,d_085_h_04,216,85.0,4.0,216
2045,d_085_h_05,229,85.0,5.0,229
2046,d_085_h_06,254,85.0,6.0,254
2047,d_085_h_07,246,85.0,7.0,246
2048,d_085_h_08,240,85.0,8.0,240
2049,d_085_h_09,236,85.0,9.0,236


Output to xlsx

In [12]:
output_df = df.drop(columns = ["carbon_intensity","day","hour"])
output_df = output_df.set_index("group")
output_df

,ref value
group,
d_000_h_00,75.0
d_000_h_01,68.0
d_000_h_02,65.0
d_000_h_03,57.0
d_000_h_04,55.0
...,...
d_364_h_19,127
d_364_h_20,97
d_364_h_21,92


In [13]:
output_filename = "./datasets/carbon_intensity_input.xlsx"
output_df.to_excel(output_filename)